<a href="https://colab.research.google.com/github/TheThreeRs/DataStudy/blob/main/TG_ZeroToAdvancedAMLFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup TigerGraph Server

> Note: If this is your first time using TigerGraph Cloud, checkout the this [blog](https://www.tigergraph.com/blog/getting-started-with-tigergraph-3-0/) to learn the basics of deploying an instance on tgcloud.io

For this workshop you can choose `Fraud and Money Laundering Detection (Fin. Services) v3` as the starter kit.

![MR](https://miro.medium.com/max/1400/1*8ZGz7MccIpFYSaqO1bszyQ.png)

Once you have the box provisioned with the Starter Kit mentioned above open GraphStudio.

![GS](https://miro.medium.com/max/1400/1*D9Ya_eBWstx-xv_i-9gbQw.png)

On the top right you will see `Global View`. Click on that and choose `MyGraph`. Once you select `MyGraph`, Global View will go away and it will look simliar to this.

![GS2](https://miro.medium.com/max/1400/1*d6hPhuJE7qttBhCk37cTWA.png)

Perfect. Now you Graph is UP. Let's move on. 

### Grab and Import Packages

In [ ]:
# FETCH PACKAGES
!pip install pyTigerGraphBeta
import pyTigerGraphBeta as tg
import pandas as pd

### Setup Connection to TigerGraph

In [ ]:
# SETUP TIGERGRAPH CONNECTION
conn = tg.TigerGraphConnection(host="https://vlad-ml3.i.tgcloud.io", graphname="AntiFraud", username="tigergraph", password="tigergraph")
# be mindful of the trailing '/' after the domain ...no dash after xyz.i.tgcloud.io

In [ ]:
# Quick Connection Test
print(conn.gsql('''LS''', options=[]))

*Create API token in TGCloud prior to next steps

In [ ]:
conn.apiToken = conn.getToken("7scbctvbo8un1d8t7i2aeq9pnd7b1p1d")

## Basic Queries


*Map and Load Data prior to query execution

In [ ]:
# Testing to see if all is well
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE OR REPLACE QUERY Test() FOR GRAPH AntiFraud { 
 
  PRINT "test"; 
}
''', options=[]))

In [ ]:
# FETCH DATA to CHECK FOR LOADED VERTICES
print(conn.gsql('''
USE GRAPH AntiFraud
INTERPRET  QUERY () FOR GRAPH AntiFraud { 
  seed = {User.*};
  S1 = SELECT s FROM seed:s LIMIT 3; 
  PRINT S1; 
}
''', options=[]))

In [ ]:
#First Query 
print(conn.gsql('''
CREATE OR REPLACE QUERY selectUser(VERTEX<User> user_id) FOR GRAPH AntiFraud { 
  seed = {user_id};
  S1 = SELECT s FROM seed:s; 
  PRINT S1; 
}
''', options=[]))

In [ ]:
# CREATE/INSTALL QUERY FOR USER TRANSACTIONS
print(conn.gsql('''
CREATE OR REPLACE QUERY selectUserTx(VERTEX<User> user_id) FOR GRAPH AntiFraud { 
  ListAccum<EDGE> @@txSend, @@txRecieve;
  seed = {user_id};
 
  SendTx = SELECT tgt FROM seed:s -(User_Transfer_Transaction:e)-> Transaction:tgt
          // WHERE s.user_id == user_id
           ACCUM @@txSend +=  e;
 
  RecieveTx = SELECT tgt FROM seed:s -(User_Recieve_Transaction:e)-> Transaction:tgt
            //  WHERE s.id == acct
              ACCUM @@txRecieve += e;
 
  PRINT @@txSend, @@txRecieve; 
}
''', options=[]))

# DML / Query Capability

In [ ]:
# Add Account Node and Edges
print(conn.gsql('''
USE GRAPH AntiFraud 

DROP JOB add_Account_and_edges

CREATE SCHEMA_CHANGE JOB add_Account_and_edges FOR GRAPH AntiFraud{
  //ALTER VERTEX User ADD ATTRIBUTE (pagerank FLOAT); 
  ADD VERTEX Account (PRIMARY_ID id STRING, cnt_tx INT) WITH primary_id_as_attribute="true";
  ADD UNDIRECTED EDGE User_to_Account (FROM User, TO Account);
  ADD UNDIRECTED EDGE Account_to_Transaction (FROM Account, TO Transaction);
   
}
RUN SCHEMA_CHANGE JOB add_Account_and_edges
''', options=[]))

*Now Map Data to the Account and User_to_Account edge in TG Studio AND Load Data if not done already


In [ ]:
# CREATE QUERY TO INSERT AND ADD Account/Account_to_Transaction Edge
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE OR REPLACE QUERY PerformInsertEnrich(/* Parameters here */) FOR GRAPH AntiFraud { 
  SumAccum<DOUBLE> @s_sumAmt, @r_sumAmt;
  SumAccum<DOUBLE> @s_txCnt, @r_txCnt;
  MinAccum<DOUBLE> @s_minAmt, @r_minAmt;
  MaxAccum<DOUBLE> @s_maxAmt, @r_maxAmt;
  AvgAccum @s_avgAmt, @r_avgAmt;
  SumAccum<INT> @sums;
  MapAccum<UINT, UINT> @@txmap;
  SetAccum<VERTEX <Transaction>> @trans; 
  
  //Goal is to insert the Account_to_Transaction edges 
  //AND add the # of transactions for each user in the corresponding user account vertex

  Seed = {User.*}; //pick up ALL users in the graph
  
  acctSend = SELECT s FROM Seed:s -((User_Transfer_Transaction|User_Recieve_Transaction):e)-> Transaction:tgt  //Traverse along 2 edges
             ACCUM 
                  s.@sums +=1,  //sum up the number of transactions for each person 
                  s.@trans += tgt; //storing each transaction record in a set
  
  acctTx = Select t FROM acctSend:s -(User_to_Account:e)- Account:t
              ACCUM 
                  t.@trans = s.@trans,
                  t.@sums = s.@sums //need to assign sum from previous query to this one for post-accum
              POST-ACCUM   
                  t.cnt_tx = t.@sums, //has to be same vertex type to perform the insert here
              FOREACH i in t.@trans DO
                  INSERT INTO Account_to_Transaction VALUES(t,i)  //iterate through our list structure and make edges
              END;
  
 // PRINT acctSend[acctSend.@sums];
 // PRINT acctTx[acctTx.@sums];
  PRINT "Edges Inserted, Accounts Updated";
}
''', options=[]))

*note Make pageRank attribute + publish before Installing queries

INSTALL THE QUERY- Interpreted mode will NOT work for this query (until future release)

## Queries to Install for Feature Generation

In [ ]:
# CREATE QUERY FOR GENERATING FEATURES USING PAGERANK
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE OR REPLACE QUERY pageRank (STRING v_type, STRING e_type,
 FLOAT max_change=0.001, INT max_iter=25, FLOAT damping=0.8, INT top_k = 100,
 BOOL print_accum = TRUE, STRING result_attr =  "", STRING file_path = "",
 BOOL display_edges = FALSE) FOR GRAPH AntiFraud{
/*
 Compute the pageRank score for each vertex in the GRAPH
 In each iteration, compute a score for each vertex:
     score = (1-damping) + damping*sum(received scores FROM its neighbors).
 The pageRank algorithm stops when either of the following is true:
 a) it reaches max_iter iterations;
 b) the max score change for any vertex compared to the last iteration <= max_change.
 v_type: vertex types to traverse          print_accum: print JSON output
 e_type: edge types to traverse            result_attr: INT attr to store results to
 max_iter; max #iterations                 file_path: file to write CSV output to
 top_k: #top scores to output              display_edges: output edges for visualization
 max_change: max allowed change between iterations to achieve convergence
 damping: importance of traversal vs. random teleport

 This query supports only taking in a single edge for the time being (8/13/2020).
*/
	TYPEDEF TUPLE<VERTEX Vertex_ID, FLOAT score> Vertex_Score;
	HeapAccum<Vertex_Score>(top_k, score DESC) @@topScores;
	MaxAccum<FLOAT> @@max_diff = 9999;    # max score change in an iteration
	SumAccum<FLOAT> @recvd_score = 0; # sum of scores each vertex receives FROM neighbors
	SumAccum<FLOAT> @score = 1;           # initial score for every vertex is 1.
	SetAccum<EDGE> @@edgeSet;             # list of all edges, if display is needed
	FILE f (file_path);

# PageRank iterations	
	Start = {v_type};                     # Start with all vertices of specified type(s)
	WHILE @@max_diff > max_change LIMIT max_iter DO
			@@max_diff = 0;
			V = SELECT s
				FROM Start:s -(e_type:e)-> v_type:t
				ACCUM t.@recvd_score += s.@score/(s.outdegree(e_type)) 
				POST-ACCUM s.@score = (1.0-damping) + damping * s.@recvd_score,
						   s.@recvd_score = 0,
						   @@max_diff += abs(s.@score - s.@score');
	END; # END WHILE loop

# Output
	IF file_path != "" THEN
	  f.println("Vertex_ID", "PageRank");
	END;

	V = SELECT s FROM Start:s
		POST-ACCUM 
			IF result_attr != "" THEN s.setAttr(result_attr, s.@score) END,
			IF file_path != "" THEN f.println(s, s.@score) END,
			IF print_accum THEN @@topScores += Vertex_Score(s, s.@score) END;

	IF print_accum THEN
		PRINT @@topScores;
		IF display_edges THEN
			PRINT Start[Start.@score];
			Start = SELECT s
					FROM Start:s -(e_type:e)-> v_type:t
					ACCUM @@edgeSet += e;
		   PRINT @@edgeSet;
		END;
	END;
}
''', options=[]))

In [ ]:
# CREATE/INSTALL QUERY FOR ACCOUNTS TOP PAGERANK SCORES (run this after having run main pageRank query)
print(conn.gsql('''
USE GRAPH AntiFraud
CREATE OR REPLACE QUERY selectTopPageRank() FOR GRAPH AntiFraud { 
  seed = {User.*};
  S1 = SELECT s FROM seed:s ORDER BY s.pagerank DESC LIMIT 25;
  PRINT S1; 
}
''', options=[]))

In [ ]:
# INSTALL QUERY FOR GENERATING FEATURES ABOUT ACCOUNT ACTIVITY
print(conn.gsql('''INSTALL QUERY PerformInsertEnrich, pageRank, selectTopPageRank''', options=[]))

In [ ]:
# INSTALL QUERY FOR GENERATING FEATURES USING PAGERANK (to install only this query)
print(conn.gsql('''INSTALL QUERY pageRank''', options=[]))


## Queries for GraphStudio
Select Account Query
```
CREATE OR REPLACE QUERY selectUser(STRING user_id) FOR GRAPH AntiFraud { 
  seed = {user_id};
  S1 = SELECT s FROM seed:s; 
  PRINT S1; 
}
```
Select top PageRank Query
```
CREATE OR REPLACE QUERY selectTopPageRank() FOR GRAPH AntiFraud { 
  seed = {User.*};
  S1 = SELECT s FROM seed:s ORDER BY s.pagerank DESC LIMIT 15;
  PRINT S1; 
}
```
Select Account Transactions
```
CREATE OR REPLACE QUERY selectAccountTx(VERTEX<User> user_id) FOR GRAPH AntiFraud { 
  ListAccum<EDGE> @@txSend, @@txRecieve;
  seed = {user_id};
  
  SendTx = SELECT tgt FROM seed:s -(User_Transfer_Transaction:e)-> Transaction:tgt
          // WHERE s.user_id == user_id
           ACCUM @@txSend +=  e;
  
  RecieveTx = SELECT tgt FROM seed:s -(User_Recieve_Transaction:e)-> Transaction:tgt
            //  WHERE s.id == acct
              ACCUM @@txRecieve += e;
  
  PRINT @@txSend, @@txRecieve; 

}
```

In [ ]:
# INSTALL ALL QUERIES
print(conn.gsql('''INSTALL QUERY ALL''', options=[]))

In [ ]:
# RUN INSTALLED QUERY selectAccount
print(conn.runInstalledQuery("selectTopPageRank", {}))

In [ ]:
# RUN INSTALLED QUERY selectAccountTx
print(conn.runInstalledQuery("selectUserTx", {"user_id": "8192"}))

# WARNING: DROP ALL - Will Delete everything in your graph!
Use this if you would like to start the notebook lab from the beginning.

In [ ]:
#print(conn.gsql('''DROP ALL''', options=[]))